In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/titanic/train.csv')
data

In [ ]:
data.isnull().sum()

In [ ]:
import seaborn as sns
sns.heatmap(data.isnull())

In [ ]:
#data.Age = data.Age.fillna(0)
#data.Cabin = data.Cabin.fillna(0)
#data.Embarked = data.Embarked.fillna(0)

In [ ]:
data = data.drop(columns = ['Name','PassengerId','Ticket','Cabin'])

In [ ]:
data['Sex'].replace( 'female', 0, inplace=True )
data['Sex'].replace( 'male', 1, inplace=True )
data['Embarked_S'] = data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
data['Embarked_Q'] = data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)
data['Embarked_C'] = data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)

In [ ]:
data

In [ ]:
x = data.iloc[:,0:12]
y = data['Survived']

In [ ]:
x

In [ ]:
x = data.drop(columns = ['Survived','Embarked'])

In [ ]:
x

In [ ]:
y

In [ ]:
x.head()

In [ ]:
x.iloc[:,4].unique()

In [ ]:
nan_cols=[2,6,7,8]   
num_cols = [0,1,3,4]
bina_cols = [5]

In [ ]:
from sklearn.preprocessing import StandardScaler,Binarizer
from sklearn.impute import KNNImputer
from sklearn.compose import make_column_transformer
from sklearn import set_config
trans  = make_column_transformer((StandardScaler(),num_cols),
                                 (KNNImputer(),nan_cols),
                                 (Binarizer(threshold = 50),bina_cols),
                                  remainder = 'passthrough')
set_config(display = 'diagram')
trans

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25)  

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
model = KNeighborsClassifier(9)
pipe = make_pipeline(trans,model)

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred = pipe.predict(x_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_test)*100

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(pipe,x_test,y_test)


In [ ]:
import seaborn as sns
sns.countplot(y)

In [ ]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
over = RandomOverSampler()
o_x,o_y = over.fit_resample(x,y)
o_y.value_counts()

In [ ]:
from imblearn.pipeline import make_pipeline
s = SMOTE()
s_pipe = make_pipeline(trans,s,model)
s_pipe

In [ ]:
s_pipe.fit(x_train,y_train)

In [ ]:
pred_s = s_pipe.predict(x_test)

In [ ]:
accuracy_score(pred_s,y_test)*100

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF= RandomForestClassifier(n_estimators = 600,max_depth=3,max_features = 'sqrt',min_samples_split = 3,min_samples_leaf =2,criterion ='entropy',min_weight_fraction_leaf = 0.1)
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline
pipe_RF = make_pipeline(trans,s,RF)

In [ ]:
pipe_RF.fit(x_train,y_train)

In [ ]:
pred_RF = pipe.predict(x_test)

In [ ]:
accuracy_score(pred_RF,y_test)*100

In [ ]:
import xgboost as xgb
model_xgbc = xgb.XGBClassifier(learning_rate = 0.01,max_depth = 8,gamma = 3)
pipe_xgbc = make_pipeline(trans,model_xgbc)
pipe_xgbc.fit(x_train,y_train)
pred_xgbc = pipe_xgbc.predict(x_test)
accuracy_score(pred_xgbc,y_test)*100

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data

In [ ]:
test_data = test_data.drop(columns = ['PassengerId','Name','Ticket','Cabin'])


In [ ]:
test_data.isnull().sum()

In [ ]:
test_data.dtypes

In [ ]:
test_data['Sex'] = test_data['Sex'].map({'female':1,'male':0}).astype(int)
test_data['Embarked_S'] = test_data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
test_data['Embarked_Q'] = test_data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)
test_data['Embarked_C'] = test_data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)

In [ ]:
#test_data.Age = test_data.Age.fillna(0)
#test_data.Fare = test_data.Fare.fillna(0)

In [ ]:
test_data.drop(columns = ['Embarked'],inplace = True)

In [ ]:
test_data

In [ ]:
test_data.iloc[:,5].unique()

In [ ]:
nan_cols=[2,5]   
num_cols = [0,1,3,4]
#bina_cols = [5]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn import set_config
trans  = make_column_transformer((StandardScaler(),num_cols),
                                 (SimpleImputer(),nan_cols),
                                  remainder = 'passthrough')
set_config(display = 'diagram')
trans

In [ ]:
pred_xgbc = pipe_xgbc.predict(test_data)
pred_xgbc

In [ ]:
submit_data = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')


In [ ]:
submit_data['Survived'] = pred_xgbc
submit_data.to_csv('submission.csv',index=False)

In [ ]:
submit_data.Survived.value_counts()